In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_excel('/content/drive/MyDrive/NNFL-Assignment2/data5.xlsx', header= None)
df.head()

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [7]:
df = df.drop_duplicates(keep='first')
df = df.sample(frac = 1, random_state = 7).reset_index(drop=True)
m = df.shape[0]
n = df.shape[1] - 1
print(m,n)
df.head()

210 7


,0,1,2,3,4,5,6,7
0,18.94,16.49,0.8750,6.445,3.639,5.0640,6.362,2
1,12.88,13.50,0.8879,5.139,3.119,2.3520,4.607,1
2,18.36,16.52,0.8452,6.666,3.485,4.9330,6.448,2
3,15.88,14.90,0.8988,5.618,3.507,0.7651,5.091,1
4,13.22,13.84,0.8680,5.395,3.070,4.1570,5.088,1


In [9]:
datan = df.values
X = datan[:, 0:7]
Y = datan[:, -1:]
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)
print(X.shape, Y.shape)

(210, 7) (210, 1)


In [26]:
def elm(Xtr, ytr, num_hidden_neurons, seed):
  
  np.random.seed(seed)
  num_features = Xtr.shape[1]                          
  V = np.random.randn(num_features, num_hidden_neurons)    
  b = np.random.randn(1, num_hidden_neurons)               
  hypo = np.dot(Xtr, V) + b                            
  H = np.tanh(hypo)                                        
  p_inv = np.linalg.pinv(H)                                
  W = np.dot(p_inv, ytr)                               

  return V, b, W

In [27]:
def encoder(Y):
  Y_hot = pd.get_dummies(Y[0]).to_numpy()
  return Y_hot

In [28]:
def tanh(x):
  return np.tanh(x)

def gaussian(x):
  return np.exp((-x)**2)

In [29]:
def normalize(D, min_features, max_features):
  Z = np.empty(D.shape)
  for i in range(D.shape[1]):
    Z[:,i] = (D[:,i] - min_features[i])/(max_features[i] - min_features[i])
  return Z

In [30]:
def normalize2(X):
  norm_X = (X - X.mean(axis = 0))/(X.std(axis = 0))
  return norm_X

In [31]:
def calculate_accuracy(Y_act, Y_pred):
  tp = tn = fn = fp = 0
  Y_act[0] = Y_act[0].astype(int)
  
  for i in range(len(Y_act)):
    if(Y_act.iloc[i,0] == 0):
      if(Y_pred.iloc[i,0] == 0):
        tp = tp + 1
      else:
        fn = fn + 1

    else:
      if(Y_pred.iloc[i,0] == 0):
        fp = fp + 1
      else:
        tn = tn + 1
  
  accuracy = (tp + tn)/len(Y_act)
  return accuracy

In [32]:
def k_cross_validation(X, Y, k, num_hidden_layer_neurons):
  a = 0
  b = round(len(Y)/k)
  Y_hot_enc = encoder(Y)
  X_arr = np.array(X)

  accuracy = []

  for i in range(k):
    if(i != k-1):                                                     
      Xts = X_arr[a:b, :]
      yts = Y.iloc[a:b, :].reset_index(drop = True)
      Xtr = np.concatenate((X_arr[:a,:], X_arr[b:,:]), axis = 0)
      ytr = np.concatenate((Y_hot_enc[:a,:], Y_hot_enc[b:,:]), axis = 0)

    else:                                                            
      Xts = X_arr[a:,:] 
      yts = Y.iloc[a:,:].reset_index(drop = True)
      Xtr = X_arr[:a,:]
      ytr = Y_hot_enc[:a,:]

    Xtr_norm = normalize2(Xtr)                    
    Xts_norm = normalize2(Xts)

    V1, b1, W1 = elm(Xtr_norm, ytr, num_hidden_neurons = num_hidden_layer_neurons, seed = 1)  
    hypo = np.dot(Xts_norm, V1) + b1                       
    H = np.tanh(hypo)                                         
    Yp = np.dot(H, W1)                                        
    Y_pred = pd.DataFrame(Yp.argmax(axis = 1))                
    acc = calculate_accuracy(yts, Y_pred)
    print("Accuracy in iteration %d is %.2f" %(i, acc*100)+ " %")
    accuracy.append(acc)
    a = a + round(len(Y)/k)
    b = b + round(len(Y)/k)

In [33]:
k_cross_validation(X, Y, 5, num_hidden_layer_neurons=397)

Accuracy in iteration 0 is 71.43 %
Accuracy in iteration 1 is 69.05 %
Accuracy in iteration 2 is 66.67 %
Accuracy in iteration 3 is 54.76 %
Accuracy in iteration 4 is 73.81 %
